In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import argparse
import onnxruntime as rt
import numpy as np
from inference_trt import init_decoder_inputs

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

SystemError: <built-in function __import__> returned a result with an error set

In [2]:
import models
from inference import checkpoint_from_distributed, unwrap_distributed, load_and_setup_model, prepare_input_sequence
from tacotron2_common.utils import to_gpu, get_mask_from_lengths

In [3]:
encoder = rt.InferenceSession('model_onnx/encoder.onnx')
decoder = rt.InferenceSession('model_onnx/decoder_iter.onnx')
postnet = rt.InferenceSession('model_onnx/postnet_2.onnx')
waveglow = rt.InferenceSession('model_onnx/waveglow.onnx')

In [4]:
# utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')

In [4]:
# print(encoder.get_inputs()[0].name)
# print(encoder.get_inputs()[1].name)
# print(encoder.get_outputs())
for out in postnet.get_outputs():
    print("this is the name of out put " + out.name)

this is the name of out put mel_outputs_postnet


In [5]:
# decoder_input = {}
for input in postnet.get_inputs():
    print("this is the name of in put " + input.name)
    

this is the name of in put mel_outputs


In [13]:
postnet.get_inputs()[0].name

'mel_outputs'

In [14]:
waveglow.get_inputs()[0].name

'mel'

In [4]:
text = "Hello world, I missed you so much."

In [5]:
## Encoder 
input_length = [50]
sequences, lengths = prepare_input_sequence([text],cpu_run = True)
sequences =  np.int64(sequences)
while len(sequences) < input_length[0]:
    sequences = np.append(sequences,0)
sequences = np.expand_dims(sequences,0)
memory, processed_memory, lens = encoder.run(["memory","processed_memory","lens"],{"sequences":sequences,"sequence_lengths":input_length})
memory, processed_memory, lens = torch.tensor(memory),torch.tensor(processed_memory),torch.tensor(lens)

In [6]:
## Decoder_iter
mel_lengths = np.zeros([memory.size(0)], dtype=np.int32)
not_finished = np.ones([memory.size(0)], dtype=np.int32)
mel_outputs, gate_outputs, alignments = (np.zeros(1), np.zeros(1), np.zeros(1))


(decoder_input, attention_hidden, attention_cell, decoder_hidden,
     decoder_cell, attention_weights, attention_weights_cum,
     attention_context, memory, processed_memory,
     mask) = init_decoder_inputs(memory, processed_memory, torch.tensor(input_length))

(decoder_input, attention_hidden, attention_cell, decoder_hidden,decoder_cell, attention_weights, attention_weights_cum,attention_context, memory, processed_memory,mask)  = decoder_input.numpy(), attention_hidden.numpy(), attention_cell.numpy(), decoder_hidden.numpy(),decoder_cell.numpy(), attention_weights.numpy(), attention_weights_cum.numpy(),attention_context.numpy(),memory.numpy(), processed_memory.numpy(),mask.numpy()

# decoder_output_name = ["decoder_output", "gate_prediction","out_attention_hidden", "out_attention_cell","out_decoder_hidden", "out_decoder_cell","out_attention_weights", "out_attention_weights_cum","out_attention_context"]
# decoder_input_element = {"decoder_input":decoder_input, "attention_hidden": attention_hidden, "attention_cell":attention_cell, "decoder_hidden":decoder_hidden ,"decoder_cell":decoder_cell, "attention_weights":attention_weights, "attention_weights_cum":attention_weights_cum,"attention_context":attention_context, "memory":memory, "processed_memory":processed_memory,"mask":mask}

# (mel_output, gate_output,
#              attention_hidden, attention_cell,
#              decoder_hidden, decoder_cell,
#              attention_weights, attention_weights_cum,
#              attention_context) = decoder.run(decoder_output_name , decoder_input_element)

# mel_outputs = np.expand_dims(mel_output, 2)
# gate_outputs = np.expand_dims(gate_output, 2)
# alignments = np.expand_dims(attention_weights, 2)

In [7]:
gate_threshold = 0.6
max_decoder_steps = 1000
first_iter = True

while True:
    decoder_output_name = ["decoder_output", "gate_prediction","out_attention_hidden", "out_attention_cell","out_decoder_hidden", "out_decoder_cell","out_attention_weights", "out_attention_weights_cum","out_attention_context"]
    decoder_input_element = {"decoder_input":decoder_input, "attention_hidden": attention_hidden, "attention_cell":attention_cell, "decoder_hidden":decoder_hidden ,"decoder_cell":decoder_cell, "attention_weights":attention_weights, "attention_weights_cum":attention_weights_cum,"attention_context":attention_context, "memory":memory, "processed_memory":processed_memory,"mask":mask}

    (mel_output, gate_output,
                 attention_hidden, attention_cell,
                 decoder_hidden, decoder_cell,
                 attention_weights, attention_weights_cum,
                 attention_context) = decoder.run(decoder_output_name , decoder_input_element)

    if first_iter:
        mel_outputs = np.expand_dims(mel_output, 2)
        gate_outputs = np.expand_dims(gate_output, 2)
        alignments = np.expand_dims(attention_weights, 2)
        first_iter = False
    else:
        mel_outputs = np.concatenate((mel_outputs, np.expand_dims(mel_output, 2)), 2)
        gate_outputs = np.concatenate((gate_outputs, np.expand_dims(gate_output, 2)), 2)
        alignments =np.concatenate((alignments, np.expand_dims(attention_weights, 2)), 2)
        
    dec = torch.le(torch.sigmoid(torch.Tensor(gate_output)), gate_threshold).to(torch.int32).squeeze(1)
    not_finished = not_finished*dec.numpy()
    mel_lengths += not_finished
    
    if np.sum(not_finished) == 0:
            print("Stopping after ",mel_outputs.size," decoder steps")
            break
    if mel_outputs.size == max_decoder_steps:
        print("Warning! Reached max decoder steps")
        break
        
    decoder_input = mel_output

Stopping after  15200  decoder steps


In [8]:
#postnet
mel_outputs_postnet = postnet.run(['mel_outputs_postnet'],{'mel_outputs':mel_outputs})

In [15]:
mel_outputs_postnet[0]

array([[[-7.46 , -7.316, -7.152, ..., -9.195, -8.65 , -7.895],
        [-6.93 , -6.656, -6.53 , ..., -7.87 , -7.582, -7.195],
        [-6.492, -6.03 , -5.797, ..., -6.797, -6.695, -6.62 ],
        ...,
        [-8.016, -7.35 , -6.59 , ..., -9.54 , -9.48 , -9.41 ],
        [-7.84 , -6.88 , -6.113, ..., -9.51 , -9.445, -9.4  ],
        [-7.95 , -6.68 , -5.824, ..., -9.53 , -9.45 , -9.414]]],
      dtype=float16)

In [ ]:
# waveglow1 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
# waveglow1 = waveglow.remove_weightnorm(waveglow)
# waveglow1 = waveglow.to('cuda')
# waveglow1.eval()

In [ ]:
# torch.Tensor(mel_outputs_postnet[0]).to('cuda')

In [ ]:
# audio = waveglow.infer(torch.Tensor(mel_outputs_postnet[0]).to('cuda'))

In [19]:
for out in waveglow.get_outputs():
    print("this is the name of output " + out.name)

this is the name of output audio


In [20]:
for out in waveglow.get_inputs():
    print("this is the name of input " + out.name)

this is the name of input mel
this is the name of input z


In [9]:
stride = 256 # value from waveglow upsample
n_group = 8
z_size2 = (mel_outputs_postnet[0].shape[2]*stride)//n_group
z = np.random.randn(1, n_group, z_size2)

In [10]:
z_size2

6080

In [11]:
z.astype('float32')

array([[[-1.1780902 , -0.21584557,  0.10846478, ...,  1.9971503 ,
         -0.41553265,  0.68270665],
        [ 1.9907577 , -0.33019024, -0.55217797, ..., -0.8074544 ,
          1.3907769 , -1.2783878 ],
        [-0.36298606,  0.2617601 ,  0.31644365, ..., -0.6730054 ,
         -0.78688556,  0.72067267],
        ...,
        [-0.51421297, -1.9321641 , -0.2196099 , ..., -0.37222224,
         -0.20373537,  0.7901144 ],
        [-0.3157504 , -1.8166993 , -0.14248283, ..., -0.21596335,
          2.595914  , -0.8115492 ],
        [-0.8377177 , -0.1688034 ,  0.772376  , ...,  1.1819868 ,
          0.12784219,  1.1712397 ]]], dtype=float32)

In [12]:
audio = waveglow.run(['audio'],{'mel':mel_outputs_postnet[0].astype('float32'),'z':z.astype('float32')})

In [15]:
audio_numpy = np.squeeze(audio[0])
audio_numpy.shape

(48640,)

In [26]:
rate = 22050
from IPython.display import Audio
Audio(audio_numpy, rate=rate)

In [13]:
import sounddevice as sd

In [17]:
sd.play(audio_numpy,samplerate=22505)

In [40]:
audio[0][0][0]

0.0020950062

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import argparse
import onnxruntime as rt
import numpy as np
from inference_trt import init_decoder_inputs
import sounddevice as sd

import models
from inference import checkpoint_from_distributed, unwrap_distributed, load_and_setup_model, prepare_input_sequence
from tacotron2_common.utils import to_gpu, get_mask_from_lengths

text = input("please enter your text: ")

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

SystemError: <built-in function __import__> returned a result with an error set

In [ ]:

# Load model.onnx form directory
encoder = rt.InferenceSession('model_onnx/encoder.onnx') 
decoder = rt.InferenceSession('model_onnx/decoder_iter.onnx')
postnet = rt.InferenceSession('model_onnx/postnet_2.onnx')
waveglow = rt.InferenceSession('model_onnx/waveglow.onnx')

## Encoder 
input_length = [50]
sequences, lengths = prepare_input_sequence([text],cpu_run=True)
sequences =  np.int64(sequences.cpu())
while len(sequences) < input_length[0]:
    sequences = np.append(sequences,0)
sequences = np.expand_dims(sequences,0)
memory, processed_memory, lens = encoder.run(["memory","processed_memory","lens"],{"sequences":sequences,"sequence_lengths":input_length})
memory, processed_memory, lens = torch.tensor(memory),torch.tensor(processed_memory),torch.tensor(lens)

## Decoder_iter
mel_lengths = np.zeros([memory.size(0)], dtype=np.int32)
not_finished = np.ones([memory.size(0)], dtype=np.int32)
mel_outputs, gate_outputs, alignments = (np.zeros(1), np.zeros(1), np.zeros(1))


(decoder_input, attention_hidden, attention_cell, decoder_hidden,
     decoder_cell, attention_weights, attention_weights_cum,
     attention_context, memory, processed_memory,
     mask) = init_decoder_inputs(memory, processed_memory, torch.tensor(input_length))

(decoder_input, attention_hidden, attention_cell, decoder_hidden,decoder_cell, attention_weights, attention_weights_cum,attention_context, memory, processed_memory,mask)  = decoder_input.numpy(), attention_hidden.numpy(), attention_cell.numpy(), decoder_hidden.numpy(),decoder_cell.numpy(), attention_weights.numpy(), attention_weights_cum.numpy(),attention_context.numpy(),memory.numpy(), processed_memory.numpy(),mask.numpy()

gate_threshold = 0.6
max_decoder_steps = 1000
first_iter = True

while True:
    decoder_output_name = ["decoder_output", "gate_prediction","out_attention_hidden", "out_attention_cell","out_decoder_hidden", "out_decoder_cell","out_attention_weights", "out_attention_weights_cum","out_attention_context"]
    decoder_input_element = {"decoder_input":decoder_input, "attention_hidden": attention_hidden, "attention_cell":attention_cell, "decoder_hidden":decoder_hidden ,"decoder_cell":decoder_cell, "attention_weights":attention_weights, "attention_weights_cum":attention_weights_cum,"attention_context":attention_context, "memory":memory, "processed_memory":processed_memory,"mask":mask}

    (mel_output, gate_output,
                 attention_hidden, attention_cell,
                 decoder_hidden, decoder_cell,
                 attention_weights, attention_weights_cum,
                 attention_context) = decoder.run(decoder_output_name , decoder_input_element)

    if first_iter:
        mel_outputs = np.expand_dims(mel_output, 2)
        gate_outputs = np.expand_dims(gate_output, 2)
        alignments = np.expand_dims(attention_weights, 2)
        first_iter = False
    else:
        mel_outputs = np.concatenate((mel_outputs, np.expand_dims(mel_output, 2)), 2)
        gate_outputs = np.concatenate((gate_outputs, np.expand_dims(gate_output, 2)), 2)
        alignments =np.concatenate((alignments, np.expand_dims(attention_weights, 2)), 2)
        
    dec = torch.le(torch.sigmoid(torch.Tensor(gate_output)), gate_threshold).to(torch.int32).squeeze(1)
    not_finished = not_finished*dec.numpy()
    mel_lengths += not_finished
    
    if np.sum(not_finished) == 0:
            print("Stopping after ",mel_outputs.size," decoder steps")
            break
    if mel_outputs.size == max_decoder_steps:
        print("Warning! Reached max decoder steps")
        break
        
    decoder_input = mel_output
    
#postnet
mel_outputs_postnet = postnet.run(['mel_outputs_postnet'],{'mel_outputs':mel_outputs})

stride = 256 # value from waveglow upsample
n_group = 8
z_size2 = (mel_outputs_postnet[0].shape[2]*stride)//n_group
z = np.random.randn(1, n_group, z_size2)
audio = waveglow.run(['audio'],{'mel':mel_outputs_postnet[0].astype('float32'),'z':z.astype('float32')})
audio_numpy = np.squeeze(audio[0])
audio = None


sd.play(audio_numpy,samplerate=22050)
